# Generate Graph Instances

In [1]:
# It's very quick to generate a large number of graphs, so we should do this on the first node.
# What is the probability of any 3-regular graph being isomorphic to another as v becomes large?

In [2]:
import networkx as nx
import numpy as np

seed = 137
np.random.seed(seed)
# We might want more graphs, additionally we want to make sure that the graphs are saved in a way that it is easy
# to recover the data, and make sure that new graphs are non-isomorphic.
num_graphs_gen = 100
graphs = []
# We might want larger d.
d = 3

# We will scan over this parameter, it's not clear if it should scale multiplicatively, or additively.
num_qubits = 4
for _ in range(num_graphs_gen):
    graphs.append(nx.generators.random_graphs.random_regular_graph(d, num_qubits))

# Simulate QAOA Landscape for each Graph Instance

In [3]:
# As the number of qubits increases, each circuit will become increasingly time consuming to simulate.
# Additionally, the number of jobs we submit (and the quality of our estimates of the optimal parameters)
# will scale with how well we discretize our grid. The feature sizes may shrink with n, so we may want our discretization
# to be a function of n.

# But there is also the question of how we want to break up the simulation. We could hand each node a graph, and ask it
# to produce the landscape. This is nice because then no parent node has to clean up individual cost evaluations and 
# regather them. 

# Jiliac: 16nodes 8cores 90GB(per node)
# Hexadec: 12nodes 16cores 120GB(per node)
# NistQ: 10nodes 32cores 500GB(per node)
# Dell: 16nodes 24cores 180GB(per node)

# Conveniently the QASM Simulator already parallelizes across nodes, so I really just need to submit the jobs.
# It will also parallelize after some number of qubits for matrix multiplication.

In [9]:
# dispatch_jobs.py
from subprocess import call
import hashlib

# def write_graph(graph, attributes={}):
#     h = hashlib.md5()
#     arr = adjacency_matrix(graph).toarray()
#     h.update(arr)
#     hash_ = h.hexdigest()
#     filename = f'{hash_}.pkl'
#     try:
#         with open(filename, 'rb') as filehandle:
#             data = dill.load(filehandle)
#             print("Fetching existing file...")
#     except FileNotFoundError:
#         data = {'graph': graph}
#     for k, v in attributes.items():
#         if data.get(k) is None:
#             data[k] = v
#         else:
#             print(f"File {filename} already has attribute {k}, not overwriting.")
#     with open(filename, 'wb') as filehandle:
#     return filename

# for graph in graphs:
#     filename = write_graph(graph)
#     cmd = f"sbatch produce_landscape.py {filename}"
#     call(cmd, shell=True)
  

In [43]:
# produce_landscape.py
import sys
def write_graph(graph, attributes={}):
    h = hashlib.md5()
    arr = adjacency_matrix(graph).toarray()
    h.update(arr)
    hash_ = h.hexdigest()
    filename = f'{hash_}.pkl'
    try:
        with open(filename, 'rb') as filehandle:
            data = dill.load(filehandle)
            print("Fetching existing file...")
    except FileNotFoundError:
        data = {'graph': graph}
    for k, v in attributes.items():
        if data.get(k) is None:
            data[k] = v
        else:
            print(f"File {filename} already has attribute {k}, not overwriting.")
    with open(filename, 'wb') as filehandle:
            data = dill.dump(data, filehandle)
    return filename

def read_graph(filename):
    with open(filename, 'rb') as filehandle:
            data = dill.load(filehandle)
    return data



from classical_optimization.qaoa_circuits import produce_gammas_betas, maxcut_qaoa_circuit
from qiskit import Aer, execute
from qiskit.providers.aer.extensions import snapshot_density_matrix

# def cost(density_matrix, num_qubits, weights):
#     rtn = 0
#     for edge, weight in weights.items():
#         rtn += .5 * weight * (1 - np.trace(Z(*edge, num_qubits).dot(density_matrix)))
#     return rtn

def cost(statevector, num_qubits, weights):
    rtn = 0
    for edge, weight in weights.items():
        rtn += .5 * weight * (1 - np.conj(statevector.T).dot(Z(*edge, num_qubits).dot(statevector)))
    return rtn

def Z(i, j, num_qubits):
    rtn = np.eye(1)
    z = np.array([[1, 0], [0, -1]])
    for k in range(num_qubits):
        if k == i or k == j:
            rtn = np.kron(rtn, z)
        else:
            rtn = np.kron(rtn, np.eye(2))
    return rtn

def weights(graph):
    rtn = {}
    for e in graph.edges:
        try:
            weight = graph.get_edge_data(e[0], e[1])['weight']
        except KeyError:
            weight = 1
        rtn[e] = weight
    return rtn

discretization = 10
max_gamma = 2*np.pi
max_beta = np.pi
gammas, betas = produce_gammas_betas(discretization, max_gamma, max_beta)

filename = sys.argv[1]
#graph = read_graph(filename)['graph']
num_qubits = len(graph.nodes)
simulator = Aer.get_backend('statevector_simulator')
experiments = []

for gamma in gammas:
        for beta in betas:
            circuit = maxcut_qaoa_circuit(gammas=[gamma], betas=[beta], p=1, num_qubits=num_qubits, weights=weights(graph), measure=False)
            experiments.append(circuit)
job = execute(experiments, backend=simulator)    
expectations = [np.real(cost(job.result().get_statevector(experiment), num_qubits=num_qubits, weights=weights(graph))) for experiment in experiments]
landscape = np.zeros((2*discretization, discretization))
for i, gamma in enumerate(gammas):
    for j, beta in enumerate(betas):
        landscape[i][j] = expectations[i*len(betas) + j]
write_graph(graph, {f"landscape_d{discretization}_b{max_beta}_g{max_gamma}": landscape})

NameError: name 'adjacency_matrix' is not defined

In [45]:
expectations

[3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.9999999999999996,
 3.0,
 3.0,
 1.287528455088411,
 1.24043495433677,
 2.936551098077402,
 3.5726995537730875,
 2.0975154501468314,
 0.9490409318152986,
 2.025364021031202,
 3.547641625409099,
 3.0,
 3.0,
 2.6870127045485765,
 2.610515491663753,
 2.896935583887079,
 3.072905450895452,
 2.847599052132718,
 2.5933810940006503,
 2.7303985224437013,
 3.032202134091259,
 3.0,
 3.0,
 2.2279576313283296,
 2.060928937983607,
 2.7749628502204784,
 3.189972918869844,
 2.6200704909017527,
 2.007135386123151,
 2.3641676861454552,
 3.101098807458269,
 3.0,
 3.0,
 1.4780192983556089,
 1.602701473042,
 3.167983839530074,
 3.586918525793352,
 2.1671306489674484,
 1.2551088077353338,
 2.358154823116447,
 3.65326052525596,
 3.0,
 3.0,
 3.407766486857202,
 3.010932721475415,
 2.4653473142277056,
 2.6727012561490233,
 3.2902999316901145,
 3.2974357588432386,
 2.6823153300647182,
 2.4615504199054135,
 3.0,
 3.0,
 3.3815335584925807,
 1.9014302521429853,
 1.

# Compute the Optimal QAOA Parameters for Each Graph

In [50]:
num_qubits = 30
num_gb = (4 * 2 * 2**num_qubits)/1E9
print(num_gb)

8.589934592
